# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 2:53PM,239161,20,ALU0007025,Alumier Labs Inc.,Released
1,Jun 8 2022 2:52PM,239160,16,ACG-2102490281,ACG North America LLC,Released
2,Jun 8 2022 2:44PM,239159,10,0085749575,ISDIN Corporation,Released
3,Jun 8 2022 2:44PM,239159,10,0085749576,ISDIN Corporation,Released
4,Jun 8 2022 2:44PM,239159,10,0085749577,ISDIN Corporation,Released
5,Jun 8 2022 2:44PM,239159,10,0085749555,ISDIN Corporation,Released
6,Jun 8 2022 2:44PM,239159,10,0085749560,ISDIN Corporation,Released
7,Jun 8 2022 2:44PM,239159,10,0085749556,ISDIN Corporation,Released
8,Jun 8 2022 2:44PM,239159,10,0085749451,ISDIN Corporation,Released
9,Jun 8 2022 2:44PM,239159,10,0085749480,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,239157,Released,24
51,239158,Released,1
52,239159,Released,42
53,239160,Released,1
54,239161,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239157,NaN,NaN,24.0
239158,NaN,NaN,1.0
239159,NaN,NaN,42.0
239160,NaN,NaN,1.0
239161,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239039,0.0,0.0,2.0
239045,29.0,0.0,3.0
239047,0.0,0.0,1.0
239050,6.0,1.0,5.0
239060,20.0,14.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239039,0,0,2
239045,29,0,3
239047,0,0,1
239050,6,1,5
239060,20,14,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239039,0,0,2
1,239045,29,0,3
2,239047,0,0,1
3,239050,6,1,5
4,239060,20,14,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239039,,,2
1,239045,29,,3
2,239047,,,1
3,239050,6,1,5
4,239060,20,14,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 2:53PM,239161,20,Alumier Labs Inc.
1,Jun 8 2022 2:52PM,239160,16,ACG North America LLC
2,Jun 8 2022 2:44PM,239159,10,ISDIN Corporation
44,Jun 8 2022 2:37PM,239158,19,"GCH Granules USA, Inc."
45,Jun 8 2022 2:33PM,239157,10,ISDIN Corporation
69,Jun 8 2022 2:24PM,239156,21,"NBTY Global, Inc."
70,Jun 8 2022 2:23PM,239155,10,ISDIN Corporation
71,Jun 8 2022 2:23PM,239154,10,ISDIN Corporation
86,Jun 8 2022 2:20PM,239153,16,ACG North America LLC
87,Jun 8 2022 2:20PM,239152,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 2:53PM,239161,20,Alumier Labs Inc.,,,1
1,Jun 8 2022 2:52PM,239160,16,ACG North America LLC,,,1
2,Jun 8 2022 2:44PM,239159,10,ISDIN Corporation,,,42
3,Jun 8 2022 2:37PM,239158,19,"GCH Granules USA, Inc.",,,1
4,Jun 8 2022 2:33PM,239157,10,ISDIN Corporation,,,24
5,Jun 8 2022 2:24PM,239156,21,"NBTY Global, Inc.",,,1
6,Jun 8 2022 2:23PM,239155,10,ISDIN Corporation,,,1
7,Jun 8 2022 2:23PM,239154,10,ISDIN Corporation,,,15
8,Jun 8 2022 2:20PM,239153,16,ACG North America LLC,,,1
9,Jun 8 2022 2:20PM,239152,10,ISDIN Corporation,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 2:53PM,239161,20,Alumier Labs Inc.,1,,
1,Jun 8 2022 2:52PM,239160,16,ACG North America LLC,1,,
2,Jun 8 2022 2:44PM,239159,10,ISDIN Corporation,42,,
3,Jun 8 2022 2:37PM,239158,19,"GCH Granules USA, Inc.",1,,
4,Jun 8 2022 2:33PM,239157,10,ISDIN Corporation,24,,
5,Jun 8 2022 2:24PM,239156,21,"NBTY Global, Inc.",1,,
6,Jun 8 2022 2:23PM,239155,10,ISDIN Corporation,1,,
7,Jun 8 2022 2:23PM,239154,10,ISDIN Corporation,15,,
8,Jun 8 2022 2:20PM,239153,16,ACG North America LLC,1,,
9,Jun 8 2022 2:20PM,239152,10,ISDIN Corporation,3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 2:53PM,239161,20,Alumier Labs Inc.,1,,
1,Jun 8 2022 2:52PM,239160,16,ACG North America LLC,1,,
2,Jun 8 2022 2:44PM,239159,10,ISDIN Corporation,42,,
3,Jun 8 2022 2:37PM,239158,19,"GCH Granules USA, Inc.",1,,
4,Jun 8 2022 2:33PM,239157,10,ISDIN Corporation,24,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 2:53PM,239161,20,Alumier Labs Inc.,1.0,NaN,NaN
1,Jun 8 2022 2:52PM,239160,16,ACG North America LLC,1.0,NaN,NaN
2,Jun 8 2022 2:44PM,239159,10,ISDIN Corporation,42.0,NaN,NaN
3,Jun 8 2022 2:37PM,239158,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Jun 8 2022 2:33PM,239157,10,ISDIN Corporation,24.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 2:53PM,239161,20,Alumier Labs Inc.,1.0,0.0,0.0
1,Jun 8 2022 2:52PM,239160,16,ACG North America LLC,1.0,0.0,0.0
2,Jun 8 2022 2:44PM,239159,10,ISDIN Corporation,42.0,0.0,0.0
3,Jun 8 2022 2:37PM,239158,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Jun 8 2022 2:33PM,239157,10,ISDIN Corporation,24.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3347764,122.0,22.0,13.0
102,478176,2.0,0.0,0.0
12,239083,1.0,0.0,0.0
15,717142,9.0,1.0,35.0
16,956566,3.0,1.0,0.0
18,239107,1.0,0.0,0.0
19,1434642,51.0,14.0,20.0
20,1195639,41.0,13.0,0.0
21,717428,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3347764,122.0,22.0,13.0
1,102,478176,2.0,0.0,0.0
2,12,239083,1.0,0.0,0.0
3,15,717142,9.0,1.0,35.0
4,16,956566,3.0,1.0,0.0
5,18,239107,1.0,0.0,0.0
6,19,1434642,51.0,14.0,20.0
7,20,1195639,41.0,13.0,0.0
8,21,717428,3.0,0.0,0.0
9,22,478297,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,122.0,22.0,13.0
1,102,2.0,0.0,0.0
2,12,1.0,0.0,0.0
3,15,9.0,1.0,35.0
4,16,3.0,1.0,0.0
5,18,1.0,0.0,0.0
6,19,51.0,14.0,20.0
7,20,41.0,13.0,0.0
8,21,3.0,0.0,0.0
9,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,122.0
1,102,Released,2.0
2,12,Released,1.0
3,15,Released,9.0
4,16,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21,22
Status,,,,,,,,,,
Completed,13.0,0.0,0.0,35.0,0.0,0.0,20.0,0.0,0.0,0.0
Executing,22.0,0.0,0.0,1.0,1.0,0.0,14.0,13.0,0.0,0.0
Released,122.0,2.0,1.0,9.0,3.0,1.0,51.0,41.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21,22
0,Completed,13.0,0.0,0.0,35.0,0.0,0.0,20.0,0.0,0.0,0.0
1,Executing,22.0,0.0,0.0,1.0,1.0,0.0,14.0,13.0,0.0,0.0
2,Released,122.0,2.0,1.0,9.0,3.0,1.0,51.0,41.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21,22
0,Completed,13.0,0.0,0.0,35.0,0.0,0.0,20.0,0.0,0.0,0.0
1,Executing,22.0,0.0,0.0,1.0,1.0,0.0,14.0,13.0,0.0,0.0
2,Released,122.0,2.0,1.0,9.0,3.0,1.0,51.0,41.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()